In [1]:
! curl -X DELETE "http://localhost:9200/hybrid_pdf_index"

{"acknowledged":true}

In [2]:
! curl -X GET "http://localhost:9200/_cat/indices?v=true&h=index,health,status,docs.count,store.size&s=index&pretty"

index                     health status docs.count store.size
.opensearch-observability green  open            0       208b
.plugins-ml-config        green  open            1      3.9kb
doc_hybrid_pdf_index      yellow open          473     12.5mb


In [3]:
%pip install llama-index
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch

%pip install llama-index-embeddings-ollama
%pip install ollama
%pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from llama_index.core import SimpleDirectoryReader
reader = SimpleDirectoryReader(input_dir="pdfs",recursive=True)
documents = reader.load_data()

In [5]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = 'BAAI/bge-m3'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=512, device=device)

In [8]:
embeddings = embed_model.get_text_embedding("box")
dim = len(embeddings)
print(dim)

1024


In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from os import getenv
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "http://localhost:9200")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "hybrid_pdf_index")

# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled with hybrid search pipeline
client = OpensearchVectorClient(
    endpoint=endpoint,
    index=idx,
    dim=dim,
    embedding_field=embedding_field,
    text_field=text_field,
    search_pipeline="hybrid-search-pipeline",
)

# initialize vector store
vector_store = OpensearchVectorStore(client)

In [11]:
idx

'hybrid_pdf_index'

In [12]:
from llama_index.core import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes, storage_context=storage_context, embed_model=embed_model
)

In [13]:
! curl -XGET http://localhost:9200/hybrid_pdf_index/_mapping | jq .

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1205  100  1205    0     0   433k      0 --:--:-- --:--:-- --:--:--  588k
{
  "hybrid_pdf_index": {
    "mappings": {
      "properties": {
        "content": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "embedding": {
          "type": "knn_vector",
          "dimension": 1024,
          "method": {
            "engine": "nmslib",
            "space_type": "l2",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 256,
              "m": 48
            }
          }
        },
        "metadata": {
          "properties": {
            "_node_content": {
              "type": "text",
              "fields": {
                "keyword": {
            

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
retriever = index.as_retriever(similarity_top_k=3,vector_store_query_mode=VectorStoreQueryMode.HYBRID)
text_retriveve = "ผู้ขับขี่ คือใคร"
prompt = retriever.retrieve(text_retriveve)
for r in prompt:
    print(r.metadata)
    print(r)

{'page_label': '3', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-29', 'last_modified_date': '2024-07-28'}
Node ID: 1a63df54-1933-4445-9140-2591c52c20be
Text: พ.ศ. ๒๕๒๒  “ผู้ขับขี่” หมายความว่า ผู้ขับรถ ผู้ประจ าเครื่อง
อุปกรณ์การขนส่งตามกฎหมายว่าด้วยการขนส่งตามกฎหมายว่าด้วยการขนส่ง
ผู้ลากเข็นยานพาหนะ  “คนเดินเท้า”   หมายความว่า
คนเดินและให้รวมตลอดถึงผู้ใช้เก้าอี้ล้อส าหรับคนพิการ  หรือรถส
าหรับเด็กด้วย    “เจ้าของรถ” หมายความรวมถึงผู้มีรถไว้ในครอบครองด้วย
“ผู้เก็บค่าโดยสาร ” หมายความว่าผู้ซึ่งรั...
Score:  1.000

{'page_label': '5', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-29', 'last_modified_date': '2024-07-28'}
Node ID: 9e95b4d0-9bb6-428e-b7c6-7b794744cc5d
Text:

In [15]:
from llama_index.vector_stores.opensearch import OpensearchVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings

# ตั้งค่า embed_model ที่คุณมีอยู่
Settings.embed_model = embed_model  # ใช้ embed_model ที่คุณโหลดมาแล้ว
Settings.llm = None  # ตั้งค่า LLM เป็น None เพื่อไม่ให้ใช้ OpenAI

# สร้าง query engine
query_engine = index.as_query_engine()

# ค้นหา
response = query_engine.query("ผู้ขับขี่ คือใคร")

# แสดงผลลัพธ์
print(response)

LLM is explicitly disabled. Using MockLLM.
Context information is below.
---------------------
page_label: 3
file_path: /Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf

พ.ศ. ๒๕๒๒  “ผู้ขับขี่” หมายความว่า ผู้ขับรถ ผู้ประจ าเครื่อง  
อุปกรณ์การขนส่งตามกฎหมายว่าด้วยการขนส่งตามกฎหมายว่าด้วยการขนส่ง  ผู้ลากเข็นยานพาหนะ  “คนเดินเท้า”  
หมายความว่า  คนเดินและให้รวมตลอดถึงผู้ใช้เก้าอี้ล้อส าหรับคนพิการ  หรือรถส าหรับเด็กด้วย   
“เจ้าของรถ” หมายความรวมถึงผู้มีรถไว้ในครอบครองด้วย  “ผู้เก็บค่าโดยสาร ” หมายความว่าผู้ซึ่งรับผิดชอบ
ในการเก็บค่าโดยสาร  และดูแลคนโดยสารที่อยู่ประจ ารถบรรทุกคนโดยสาร

page_label: 5
file_path: /Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf

ถึง “ผู้ขับขี่” ว่าเป็น  
ผู้ได้รับอนุญาตให้เป็นผู้ขับขี่รถนั้น  ตรงตามกฎหมาย  ตรงตามประเภทของใบอนุญาตหรือไม่  (พ.ร.บ.รถยนต์ ฯ 
ม.๔๓ ม.๔๓ ทวิ หรือ พ.ร.บ.การขนส่งทางบกฯ ม. ๙๒ ม.๙๔ และ ม.๙๕ ) เป็นต้น  
      สภาพของถนนหรือทางควรจะต้องพิจารณาก่อนว่าถนนหรือซอยหรือทางนั้นเป็น  “ทาง”  
ตามความหมายขอ